# Build lookup tables

In [1]:
%load_ext autoreload
%autoreload 2

import math
from itertools import product
import uproot
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import matplotlib
matplotlib.rcParams.update({'font.size': 12})

from calorimetry_likelihood.calo_likelihood import caloLikelihood
from calorimetry_likelihood.general_functions import selection_for_train, selection_planes, load_lookup_table
from calorimetry_likelihood.loaders import load_data
from calorimetry_likelihood.hard_coded import branches_lookup
from calorimetry_likelihood.initializers import pdgid2name, initialisation_for_caloriemtry_data_mc

## Define filename and type of separation

In [2]:
# pdg_codes = [321, 2212]
pdg_codes = [13, 2212]
input_filename = '~/Desktop/MicroBooNE/calorimetry_likelihood/v08/bnb_nu/out_mc_skimmed.root'
training_fraction = 0.2

In [3]:
output_plot_folder = '/home/nic/Desktop/MicroBooNE/calorimetry_likelihood/trial/'
output_plot_folder_pdfs = output_plot_folder + 'pdfs/'

!mkdir -p $output_plot_folder
!mkdir -p $output_plot_folder_pdfs

## Load data

In [4]:
array, pot = load_data(filename=input_filename, 
                              branches=branches_lookup, 
                              fraction=training_fraction, 
                              before=True)
initialisation_for_caloriemtry_data_mc(array)

start loading ~/Desktop/MicroBooNE/calorimetry_likelihood/v08/bnb_nu/out_mc_skimmed.root
Done!


/home/nic/miniconda3/envs/pelee/lib/python3.7/site-packages/calorimetry_likelihood/general_functions.py:39: RuntimeWarning: divide by zero encountered in true_divide
  return 0.3 / cos


## Define parameters Lookup Table

In [5]:
parameters_names = ['rr', 'pitch']
parameters_bin_edges = [[0., 2, 4, 7, 10, 15, 20, 30, 50, 100, 300, 2000],
                  [0.3, 0.6, 1, 1.5, 3, 30]]

dedx_var = 'dedx'
dedx_var_end = ''
dedx_bins = [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 9, 10, 12, 15, 20, 25, 30, 35, 40, 45, 50]

parameters = []
dedx_vars = []
for plane in ['u', 'v', 'y']:
    parameters.append([parameter_name+'_'+plane for parameter_name in parameters_names])
    dedx_vars.append(dedx_var+'_'+plane+dedx_var_end)

pdgcode_var = 'backtracked_pdg'

caloLike_train = caloLikelihood()
caloLike_train.setPdgCodeVar(pdgcode_var)

for i in [0, 1, 2]:
    caloLike_train.setVariablesBinning(i, parameters[i], parameters_bin_edges, dedx_vars[i], dedx_bins)

## Compute Lookup Table

In [6]:
caloLike_train.setArray(array, 
                        quality_mask=selection_for_train(array), 
                        quality_masks_planes=selection_planes(array))
caloLike_train.setPdgCodeVar(pdgcode_var)

for i in [0, 1, 2]:
    caloLike_train.buildLookUpTableMC(i, pdg_codes[0])    
    caloLike_train.buildLookUpTableMC(i, pdg_codes[1])    
    caloLike_train.buildLookUpLogLikelihoodRatio(i, pdg_codes)

/home/nic/Dropbox/MicroBooNE/calorimetry_likelihood/lib/calo_likelihood.py:138: RuntimeWarning: invalid value encountered in true_divide
  hist_total/hist_partial[..., np.newaxis],
/home/nic/Dropbox/MicroBooNE/calorimetry_likelihood/lib/calo_likelihood.py:149: RuntimeWarning: divide by zero encountered in log
  np.log(table_1) - np.log(table_2),
/home/nic/Dropbox/MicroBooNE/calorimetry_likelihood/lib/calo_likelihood.py:149: RuntimeWarning: invalid value encountered in subtract
  np.log(table_1) - np.log(table_2),


## Print the Lookup Table

In [8]:
caloLike_train.printCplusplusLookUp(filename=f'../dumped_objects/LLRPID_{pdgid2name[pdg_codes[0]]}_{pdgid2name[pdg_codes[1]]}_lookup.h', 
                                    pdg_codes=pdg_codes,
                                    planes=[0, 1, 2])

In [9]:
lookup_calo_filename = f'../dumped_objects/{pdgid2name[pdg_codes[0]]}_{pdgid2name[pdg_codes[1]]}_lookup.dat'

In [10]:
caloLike_train.save(filename=lookup_calo_filename)

# Plot of the pdfs in bins of residual range and pitch

In [11]:
load_lookup = False

In [12]:
if load_lookup:
    load_lookup_table(caloLike_train, lookup_calo_filename)

In [13]:
for plane_num in [0, 1, 2]:
    print(f'plane {plane_num}')
    this_folder = output_plot_folder_pdfs + f'plane{plane_num}/'
    !mkdir -p $this_folder
    bin_centers = product(*caloLike_train.parameters_bin_centers[plane_num])
    for bin_center in bin_centers:
        caloLike_train.plotLookUpDedxMC(plane_num=plane_num, 
                                  pdg_code=pdg_codes[0], 
                                  parameters_value=bin_center, 
                                  label=pdgid2name[pdg_codes[0]])
        caloLike_train.plotLookUpDedxMCfancy(plane_num=plane_num, 
                                  pdg_code=pdg_codes[1], 
                                  parameters_value=bin_center, 
                                  label=pdgid2name[pdg_codes[1]])
        plt.xlim(0, 20)
        plt.ylim(bottom=0)
        plt.tight_layout()
        save_label = f'rr_{bin_center[0]:05.0f}_pitch_{bin_center[1]:04.1f}'.replace('.', '')
        plt.savefig(this_folder + save_label + '.png', dpi=250)
        plt.savefig(this_folder + save_label + '.pdf')
        plt.close()

plane 0
plane 1
plane 2
